# Speed comparision

In [1]:
from fse.models import Average
from fse.models.average import train_average_np
from fse.models.average_inner import train_average_cy

from fse.models.average import MAX_WORDS_IN_BATCH

from fse import IndexedList

import numpy as np

import gensim.downloader as api
data = api.load("quora-duplicate-questions")

sentences = []
batch_size = 0
for d in data:
    strings = d["question1"].split()
    if len(strings) + batch_size < MAX_WORDS_IN_BATCH:
        sentences.append(strings)
        batch_size += len(strings)
sentences = IndexedList(sentences)

/Users/oliverborchers/anaconda3/envs/fsedev/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# Test W2V Model

In [2]:
import gensim.downloader as api

w2v = api.load("glove-wiki-gigaword-100")
ft = api.load("fasttext-wiki-news-subwords-300")

To test if the fast version is available, you need to import the variable FAST_VERSION from fse.models.average. 
1 : The cython version is available
-1 : The cython version is not available.

If the cython compiliation fails, you will be notified.

In [3]:
from fse.models.average import FAST_VERSION
FAST_VERSION

1

In [4]:
%%timeit
w2v_avg = Average(w2v)

300 µs ± 18.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
%%timeit
w2v_avg = Average(w2v, lang_freq="en")

363 ms ± 9.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The slowest part during the init is the induction of frequencies for words, as some pre-trained embeddings do not come with frequencies for words. This is only necessary for the SIF and uSIF Model, not for the Average model.

In [6]:
w2v_avg = Average(w2v)
statistics = w2v_avg.scan_sentences(sentences)
w2v_avg.prep.prepare_vectors(sv=w2v_avg.sv, total_sentences=statistics["max_index"], update=False)
memory = w2v_avg._get_thread_working_mem()

In [7]:
%%timeit
train_average_np(model=w2v_avg, indexed_sentences=sentences, target=w2v_avg.sv.vectors, memory=memory)

34.3 ms ± 2.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%%timeit
train_average_cy(model=w2v_avg, indexed_sentences=sentences, target=w2v_avg.sv.vectors, memory=memory)

2.94 ms ± 120 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


For 90 sentences, the Cython version is about 8-15 faster than the numpy version when using a Word2Vec type model.

In [9]:
out_w2v_np = np.zeros_like(w2v_avg.sv.vectors)
out_w2v_cy = np.zeros_like(w2v_avg.sv.vectors)
train_average_np(model=w2v_avg, indexed_sentences=sentences, target=out_w2v_np, memory=w2v_avg._get_thread_working_mem())
train_average_cy(model=w2v_avg, indexed_sentences=sentences, target=out_w2v_cy, memory=w2v_avg._get_thread_working_mem())

np.allclose(out_w2v_np, out_w2v_cy)

True

# Test FastTextModel

In [10]:
ft_avg = Average(ft)
statistics = ft_avg.scan_sentences(sentences)
ft_avg.prep.prepare_vectors(sv=ft_avg.sv, total_sentences=statistics["max_index"], update=False)
memory = ft_avg._get_thread_working_mem()

In [11]:
%%timeit
train_average_np(model=ft_avg, indexed_sentences=sentences, target=ft_avg.sv.vectors, memory=memory)

42.1 ms ± 4.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit
train_average_cy(model=ft_avg, indexed_sentences=sentences, target=ft_avg.sv.vectors, memory=memory)

4.14 ms ± 207 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


With a FastText type model, the cython routine is about 5-10 times faster.

In [13]:
out_ft_np = np.zeros_like(ft_avg.sv.vectors)
out_ft_cy = np.zeros_like(ft_avg.sv.vectors)
train_average_np(model=ft_avg, indexed_sentences=sentences, target=out_ft_np, memory=ft_avg._get_thread_working_mem())
train_average_cy(model=ft_avg, indexed_sentences=sentences, target=out_ft_cy, memory=ft_avg._get_thread_working_mem())

np.allclose(out_ft_np, out_ft_cy)

True